# PETs/TETs – Hyperledger Aries – Authority Agent (Issuing Authority) 🏛️

---
⚠️ <span style='background : yellow'>**Warning:**</span> Run the notebook only once after running `./manage.sh start` in your terminal. Before running this notebook again, execute `./manage.sh down` to delete the postgres databases (i.e., the agents' digital wallets). Otherwise, the storage will persist in the postgres database (even when executing `./manage.sh stop`).

---

In [1]:
%%javascript
document.title='🏛️ Authority'

<IPython.core.display.Javascript object>

## **PART 1: Declare Authority Agent as an Issuing Authority**


**What:** Initialize the authority agent as an issuing authority on the Sovrin StagingNet. 

**Why:** Enable the authority to certify that manufacturer1, manufacturer2, manufacturer3 are manufacturers.

**How:**
1. [Initialize authority agent and write DID to Sovrin StagingNet](#1) <br>

2. [Author schemes:](#2) design and register schemes to <br>
a. certify an agent is a city, and <br>
b. certify an agent as a manufacturer
3. [Store identifiers of schema and VC definition to access it in notebooks of other agents](#3)

**Accompanying Notebooks:** -

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import os

#from aries_cloudcontroller import AriesAgentController
from libs.aries_basic_controller import AriesAgentController
from termcolor import colored

import libs.helpers as helpers
from libs.agent_connection_manager import IssuingAuthority

#### 0.2 – Variables

Set `issue_city_schema = True` and/or `issue_manufacturer_schema = True` to issue a new Schema 1 and/or Schema 2, or update Schema 1 and/or Schema 2 with a new verion number. This is necessary if you ran `./manage.sh start` for the first time (or after running `./manage.sh down`).

In [3]:
# Indicate which schemes should be defined
issue_city_schema = True
issue_manufacturer_schema = True

# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

<a id=1> </a> 
### 1 - Initialize Authority Agent as Issuing Authority
This section initializes an ACA-PY agent and defines the agent as an IssuingAuthority (from the `AgentConnectionManager` (ACM) package) to manage the aries agent. To give the agent issuing power, we define a DID and write it to the Sovrin StagingNet. The Sovrin StagingNet is used as the underlying network to write and resolve cryptographic objects in this PoC.

#### 1.1 – Init ACA-Py agent controller and ACM issuing authority

In [4]:
agent_controller = AriesAgentController(admin_url,api_key)
print(colored("Initialising an aries agent controller with admin api at {admin_url} and an api key of {api_key}".format(admin_url=admin_url, api_key=api_key), "green", attrs=["bold"]))

authority_agent = IssuingAuthority(agent_controller)

Initialising an aries agent controller with admin api at http://authority-agent:3021 and an api key of adminApiKey
Subscribing too: connections
Subscribing too: basicmessages
Subscribing too: issue_credential
Successfully initiated AgentConnectionManager for a(n) Issuing Authority ACA-PY agent


#### 1.2 – Get agent's DID and write it to the Sovrin StagingNet
Get the DID of the authority agent (), and register it with the Sovrin StagingNet ledger.

In [5]:
# Get DID of authority agent
did_obj = authority_agent.get_did() # The method calls an existing DID, or creates a new DID if no existing DID was found
authority_did = did_obj["did"]

admin_request GET to /wallet/did/public
json: None params: {} data: None
resp_text {"result": null}
returning json
create_did:  /wallet/did/create
Controller POST %s request to Agent%s /wallet/did/create 
admin_request POST to /wallet/did/create
json: None params: {} data: None
resp_text {"result": {"did": "Mf5Yquf2DCnEmbyNb6VrYT", "verkey": "CFzd2MaWs78afLFtwEocQWVKhqetb2Pb8AGsPhkDw1Ht", "posture": "wallet_only", "key_type": "ed25519", "method": "sov"}}
returning json
Response from POST %s received: 
%s /wallet/did/create {
    "result": {
        "did": "Mf5Yquf2DCnEmbyNb6VrYT",
        "verkey": "CFzd2MaWs78afLFtwEocQWVKhqetb2Pb8AGsPhkDw1Ht",
        "posture": "wallet_only",
        "key_type": "ed25519",
        "method": "sov"
    }
}
Successfully created a new DID:
{
    'result': {
        'did': 'Mf5Yquf2DCnEmbyNb6VrYT',
        'key_type': 'ed25519',
        'method': 'sov',
        'posture': 'wallet_only',
        'verkey': 'CFzd2MaWs78afLFtwEocQWVKhqetb2Pb8AGsPhkDw1Ht',
  

In [ ]:
# Write DID to Sovrin StagingNet
authority_agent.write_did_to_ledger(did_obj)

#### 1.3 – Accept Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, a Transaction Author Agreement (TAA) must be accepted before an issuing authority has the right to write to the ledger. The TAA is accepted by signing the TAA using the DID registered on the ledger.

In [ ]:
#authority_agent.accept_taa_agreement()

#### 1.4 – Finalize initiating agent as issuing authority by making DID public

In [ ]:
authority_agent.make_did_public(did_obj)

<a id=2> </a> 
### 2 – Write VC schema to certify city-status (Schema 1) and manufacturer-status (Schema 2)


#### 2.1 – Schema 1: VCs for Cities

In [ ]:
if issue_city_schema is True:
    # Define a unique schema name on the ledger, version the schema (to be able to update it), and define attributes in the schema
    schema_name = "certify-city-agency"
    schema_version = "0.0.1"
    attributes = ["city", "country", "isCityAgency"]

    # Define schema
    schema_city_id = authority_agent.write_vc_schema(schema_name, schema_version, attributes)
    
    # Write schema credential definition transaction to the ledger to specify cryptographic material the agent uses to sign all VCs
    cred_def_city_id = authority_agent.write_vc_cred_def(schema_city_id)
          
else:
    # Get old identifiers 
    schema_city_id = fnc.get_identifiers()["city_schema_identifiers"]["schema_id"]
    cred_def_city_id = fnc.get_identifiers()["city_schema_identifiers"]["cred_def"]

# Store credentials in dictionary for storage later (to be able to access the identifiers from another jupyter notebook)
city_identifiers = {"schema_id": schema_city_id, "cred_def": cred_def_city_id}

#### 2.2 – Schema 2: VCs for Manufacturers

In [ ]:
if issue_manufacturer_schema is True:
    # Define a unique schema name on the ledger, version the schema (to be able to update it), and define attributes in the schema
    schema_name = "certify-manufacturer"
    schema_version = "0.0.1"
    attributes = ["manufacturerName", "manufacturerCountry", "manufacturerCity", "isManufacturer"]

    # Define schema
    schema_manufacturer_id = authority_agent.write_vc_schema(schema_name, schema_version, attributes)
    
    # Write schema credential definition transaction to the ledger to specify cryptographic material the agent uses to sign all VCs
    cred_def_manufacturer_id = authority_agent.write_vc_cred_def(schema_manufacturer_id)
          
else:
    # Get old identifiers 
    schema_manufacturer_id = helpers.get_identifiers()["manufacturer_schema_identifiers"]["schema_id"]
    cred_def_manufacturer_id = helpers.get_identifiers()["manufacturer_schema_identifiers"]["cred_def"]

# Store credentials in dictionary for storage later (to be able to access the identifiers from another jupyter notebook)
manufacturer_identifiers = {"schema_id": schema_manufacturer_id, "cred_def": cred_def_manufacturer_id}

<a id=3> </a> 
### 3 – Store identifiers for use throughout other agents' notebooks

The IDs of the Schema 1nd VC definitions are required whenever the Authority agent issues credentials or constrains acceptable proof requests. For notebooks, it is easier to store the value pair as a string in a cell, and load them into the jupyter store. In real applications, the values should be stored in environment variables or a database.

Thus, the identifiers are printed and copied across the main business logic notebooks and stored as variables. This process only needs to be repeated when you ran `./manage.sh start` for the first time, or executed `./manage.sh down` (instead of `./manage.sh stop`).  

In [ ]:
identifiers = {"city_schema_identifiers": city_identifiers, "manufacturer_schema_identifiers": manufacturer_identifiers, "authority_did": authority_did}
helpers.store_identifiers(dict(identifiers))

### 4 – Terminate Controller

In [ ]:
await agent_controller.terminate()